In [ ]:
# Necassary library imports
import os
import time
import logging
from tqdm import tqdm
from datetime import datetime

# Importing Modular File Functions
from api.test import test_api_connection
from task_generator.task_manager import create_task_list
from task_generator.task_manager import generate_example
from config.model import BASE_DIR

In [ ]:
# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [ ]:
# Output settings
OUTPUT_DIR = BASE_DIR / "raspberry_pi_code_examples"
TOTAL_EXAMPLES = 1000

# Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
def generate_examples():
    """Main function to orchestrate the two-stage generation process."""
    logger.info(f"Starting two-stage generation of {TOTAL_EXAMPLES} Raspberry Pi code examples")
    
    # Current user info for metadata
    current_user = os.environ.get("USER", "Unknown")
    current_datetime = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
    
    logger.info(f"Current Date and Time (UTC): {current_datetime}")
    logger.info(f"Current User's Login: {current_user}")
    
    # First test the API connection
    if not test_api_connection():
        return
    
    # Create the task list
    tasks = create_task_list(TOTAL_EXAMPLES)
    logger.info(f"Created task list with {len(tasks)} examples")
    
    # Setup progress tracking
    successful = 0
    failed = 0
    
    # Initialize progress bar
    pbar = tqdm(total=len(tasks), desc="Generating examples")
    
    # Process tasks
    start_time = time.time()
    for i, (category, subcategory, example_id) in enumerate(tasks):
        logger.info(f"Generating example {i+1}/{len(tasks)}: {category}/{subcategory} (ID: {example_id})")
        
        if generate_example(category, subcategory, example_id):
            successful += 1
        else:
            failed += 1
        
        # Update progress bar
        pbar.update(1)
        pbar.set_description(f"Generated: {successful}, Failed: {failed}")
        
        # Calculate and display ETA
        elapsed = time.time() - start_time
        avg_time_per_item = elapsed / (i + 1)
        remaining_items = len(tasks) - (i + 1)
        eta_seconds = avg_time_per_item * remaining_items
        
        if i % 10 == 0 and i > 0:
            eta_hours = int(eta_seconds // 3600)
            eta_minutes = int((eta_seconds % 3600) // 60)
            logger.info(f"Progress: {i+1}/{len(tasks)} examples. Estimated time remaining: {eta_hours}h {eta_minutes}m")
            
    pbar.close()
    
    # Final report
    end_time = time.time()
    total_time = end_time - start_time
    hours = int(total_time // 3600)
    minutes = int((total_time % 3600) // 60)
    seconds = int(total_time % 60)
    
    logger.info(f"Generation complete: {successful} successful, {failed} failed")
    logger.info(f"Total execution time: {hours}h {minutes}m {seconds}s")
    logger.info(f"Examples saved in {os.path.abspath(OUTPUT_DIR)}")  

In [ ]:
if __name__ == "__main__":
    generate_examples()